# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

In [2]:
from tqdm import tqdm
import pandas as pd

In [3]:
df=pd.read_csv('/datasets/users_behavior.csv')
df.info()
df['messages']=df['messages'].astype("int")
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(3), int64(2)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83,19915.42,0
1,85.0,516.75,56,22696.96,0
2,77.0,467.66,86,21060.45,0
3,106.0,745.53,81,8437.39,1
4,66.0,418.74,1,14502.75,0
5,58.0,344.56,21,15823.37,0
6,57.0,431.64,20,3738.90,1
7,15.0,132.40,6,21911.60,0
8,7.0,43.39,3,2538.67,1
9,90.0,665.41,38,17358.61,0


## Разбейте данные на выборки

In [4]:
features = df.drop(['is_ultra'],axis=1)#Извлекаем признаки для проверки 
target = df['is_ultra']#извлекаем целевой признак

(train_features,test_features,train_target,test_target) = train_test_split(features,target, test_size=0.4, random_state=12345)

(valid_features,test_features,valid_target,test_target) = train_test_split(test_features,test_target, test_size=0.5, random_state=12345)

train_features.shape


(1928, 4)

In [5]:
test_features.shape

(643, 4)

In [6]:
valid_features.shape

(643, 4)

## Исследуйте модели

In [7]:
best_RF = None
best_accuracy_RF = 0
best_est_RF = 0
best_depth_RF = 0
for est in tqdm(range(10,200,10)):
    for depth in range(2,25):
        RF = RandomForestClassifier(random_state = 12345,n_estimators = est, max_depth = depth)
        RF.fit(train_features,train_target)
        prediction_valid_RF = RF.predict(valid_features)
        accuracy_RF = accuracy_score(prediction_valid_RF, valid_target)
        if best_accuracy_RF < accuracy_RF:
            best_RF = RF
            best_depth_RF = depth
            best_est_RF = est
            best_accuracy_RF = accuracy_RF


print(best_accuracy_RF)

100%|██████████| 19/19 [03:46<00:00, 11.90s/it]

0.8087091757387247


In [8]:
best_DT = None
best_accuracy_DT = 0
best_depth_DT = 0
for depth in tqdm(range(2,52)):
    DT = DecisionTreeClassifier(random_state = 12345, max_depth = depth)
    DT.fit(train_features,train_target)
    prediction_valid_DT = DT.predict(valid_features)
    accuracy_DT = accuracy_score(prediction_valid_DT, valid_target)
    if best_accuracy_DT < accuracy_DT:
        best_DT = DT
        best_accuracy_DT = accuracy_DT
        best_depth_DT = depth


print(best_accuracy_DT)

100%|██████████| 50/50 [00:00<00:00, 101.17it/s]

0.7853810264385692


In [9]:
best_LR = None
best_accuracy_LR = 0
for est in tqdm(range(2,52)):
    LR = LogisticRegression(random_state = 12345,max_iter= 1000,tol=1e-5,solver = 'lbfgs' )
    LR.fit(train_features,train_target)
    prediction_valid_LR = LR.predict(valid_features)
    accuracy_LR = accuracy_score(prediction_valid_LR, valid_target)
    if best_accuracy_LR < accuracy_LR:
            best_LR = LR
            best_accuracy_LR = accuracy_LR
print(best_accuracy_LR)

100%|██████████| 50/50 [00:00<00:00, 50.04it/s]

0.7107309486780715


вывод:модель 'RandomForest' работает медленно,но точно более 80% удачных предсказаний
модель 'DecisionTree' работает намного шустрее, при этом ее предсказания не сильно отличаются от 'RandomForest' (78% правильных предсказаний)по этому я считаю, что это оптимальная для данного проекта
моделью.
модель 'LogisticRegression' самая не точная, но такая же быстрая как и 'DecisionTree', в рамках данного проекта она проходит понижней границе точности предсказаний 71%.

## Проверьте модель на тестовой выборке

In [10]:
prediction_test_DT = best_DT.predict(test_features)

accuracy_DT_test = accuracy_score(prediction_test_DT, test_target)

print("дерево решений:",accuracy_DT_test)

дерево решений: 0.7791601866251944


вывод: 
самой оптимальной моделью по скоторсти и точности является "дерево решений", его я и спользовал для проверки модели на тестовой вборке

## (бонус) Проверьте модели на адекватность

In [11]:
DC=DummyClassifier(strategy='most_frequent',random_state=12345)
DC.fit(train_features,train_target)
prediction_valid_DC = DC.predict(valid_features)
accuracy_DC = accuracy_score(prediction_valid_DC, valid_target)
print(accuracy_DC)

0.7060653188180405


In [12]:
prediction_test_DС = DC.predict(test_features)
accuracy_DС_test = accuracy_score(prediction_test_DС , test_target)
print(accuracy_DС_test)

0.6842923794712286


вывод: простая модель показазала себя хуже чем более сложная 68% верных предсказаний,маловато будет.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
